In [18]:

import numpy as np
import collections.abc 
from time import sleep


import gmaps 
import googlemaps
import re
import requests
import json
import geopandas
import cartoframes
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
load_dotenv
maps_token = getpass.getpass("maps_token: ")
map_client = googlemaps.Client(maps_token)

API queries_quota: 60


In [3]:
def extracting_data(business,keyword, radius):
    sleep(1)
    location = (41.4036522,2.189654)  
    radius = radius # radius in meters
    business = business
    keyword = keyword
    results = []
    response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)
    results.extend(response['results'])
    while True:
        try:
            sleep(2)
            response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
            results.extend(response['results'])
            next_page = response['next_page_token']
        except:
            break
    return results    

In [4]:
def dataframe(data, business,keyword):
    list_local = []
    df = pd.DataFrame(columns = ['Category','Type', 'Name', 'Address', 'Latitude', 'Longitude'])
    for result in data:
        local = {}
        local['Category'] = business
        local['Type'] = keyword
        local['Name'] = result['name']
        local['Address'] = result['vicinity']
        local['Latitude'] = result['geometry']['location']['lat']
        local['Longitude'] = result['geometry']['location']['lng']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    

In [5]:
bars = extracting_data('Bar', 'Bar', 1500)

In [6]:
restaurant = extracting_data('Restaurant', 'Restaurant', 1000)

In [49]:
school = extracting_data('School', 'public School', 3000)

In [53]:
veg_restaurant = extracting_data('Restaurant', 'vegetarian', 500)

In [8]:
df_bars = dataframe(bars,'Bar', 'Bar' )

In [9]:
df_restaurants = dataframe(restaurant,'Restaurant', 'Restaurant')

In [54]:
df_school = dataframe(school, 'School', 'School')


In [55]:
df_veg = dataframe(veg_restaurant,'Restaurant', 'vegetarian')
df_veg

,Category,Type,Name,Address,Latitude,Longitude
0,Restaurant,vegetarian,GreenVita Restaurante Glòries,"Av. Diagonal, 208, Barcelona",41.405468,2.191940
1,Restaurant,vegetarian,Honest Greens Westfield Glories,"Av. Diagonal, 208, Barcelona",41.405112,2.192273
2,Restaurant,vegetarian,Sopa Barcelona,"Carrer de Roc Boronat, 114, Barcelona",41.402313,2.195387
3,Restaurant,vegetarian,Morgentau. Restaurante Brunch Vegano Barcelona,"C/ de la Llacuna, 114, Barcelona",41.403824,2.195583
4,Restaurant,vegetarian,Debby's Good Veggie Food,"C/ de Sant Joan de Malta, 48, Local 5, Barcelona",41.407542,2.193707
5,Restaurant,vegetarian,Pure Cuisine,"Av. Diagonal, 208, Barcelona",41.405130,2.192812
6,Restaurant,vegetarian,Tierra Burrito Bar,"C/ d'Àvila, 171, Barcelona",41.401941,2.188674
7,Restaurant,vegetarian,L'Encant Vegetarià,"C. de los Castillejos, 210, Barcelona",41.403886,2.181901
8,Restaurant,vegetarian,Swad-The Indian Restaurant,"C/ de Sancho de Ávila, 167, Barcelona",41.403739,2.196475
9,Restaurant,vegetarian,Koh,"C. de Pujades, 133, Barcelona",41.399727,2.197947


In [65]:
original_map = Map(location = ['41.4034532','2.1896495'], zoom_start=15)

In [66]:
def icon(color_, icon_c, icon_):
    icon = Icon(
            color = color_,
            icon_color = icon_c,
        
            icon = icon_,
            prefix="fa"
            )
    return icon

In [67]:
icons = icon('black', 'white', 'building')

In [68]:
companie_spot = Marker(location=['41.4039759','2.1906131'], tooltip='Lessa Soluctions', icon = icons)
companie_spot.add_to(original_map)

In [69]:
original_map

In [70]:
def add_spot(data,color, icon_color, icon):
    for k, v in data.iterrows():
        name = v['Name']
        lat = v['Latitude']
        long = v['Longitude']
        
        icon__ = Icon(
            color = color,
            icon_color = icon_color,
            icon = icon,
            prefix="fa"
            )
        test_marker = Marker(location=[lat, long], tooltip=name, icon = icon__)
        test_marker.add_to(original_map)
    return original_map
#, color_, icon_color_, icon_

In [72]:
map_restaurant = add_spot(df_restaurants,'orange', 'white', 'utensils')
#, 'orange', 'white', 'utensils'

In [73]:
original_map

In [74]:
map_bars = add_spot(df_bars, 'red', 'white', 'beer-mug-empty')

In [75]:
original_map

In [76]:
map_school = add_spot(df_school, 'blue', 'black', 'graduation-cap')

In [77]:
original_map

In [78]:
map_veg = add_spot(df_veg, 'green', 'white', 'wheat-awn')

In [79]:
original_map